In [116]:
import pandas as pd
# pd.options.display.max_colwidth = 200

import numpy as np
from collections import defaultdict

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("white")

from pymystem3 import Mystem; mystem = Mystem()
from functools import lru_cache


from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

In [153]:
data = []
DATA_PATH = './data/'
for csv_name in ['recent_news.csv']:
    data.append(pd.read_csv(DATA_PATH + csv_name))
data = pd.concat(data)

In [154]:
data.head()

,date,site_id,text,title,url
0,2017-04-12 15:14:00,NOVAYA,Основатель Фонда борьбы с коррупцией (ФБК) Але...,Навальный назвал ложью объяснение Усманова о п...,https://www.novayagazeta.ru/news/2017/04/12/13...
1,2017-04-12 15:04:00,NOVAYA,Председатель комитета Госдумы по безопасности ...,В Госдуме отозвали законопроект о праве полици...,https://www.novayagazeta.ru/news/2017/04/12/13...
2,2017-04-12 14:57:00,NOVAYA,"В среду, 12 апреля, в Московском окружном воен...",Подсудимый по делу Немцова назвал критику «пра...,https://www.novayagazeta.ru/news/2017/04/12/13...
3,2017-04-12 14:46:00,NOVAYA,Депутаты Госдумы проголосовали в первом чтении...,Госдума в первом чтении проголосовала за перен...,https://www.novayagazeta.ru/news/2017/04/12/13...
4,2017-04-12 14:03:00,NOVAYA,Мосгорсуд признал законным продление ареста ещ...,Мосгорсуд оставил в силе арест Никите Белых до...,https://www.novayagazeta.ru/news/2017/04/12/13...


In [155]:
class Pipeline(object):
    def __init__(self, *args):
        self.transformations = args
    def __call__(self, x):
        res = x
        for f in self.transformations:
            res = f(res)
        return res

from nltk.corpus import stopwords
from stop_words import get_stop_words
en_sw = get_stop_words('en')
ru_sw = get_stop_words('ru')
STOP_WORDS = set(en_sw) | set(ru_sw)
STOP_WORDS = STOP_WORDS | set(stopwords.words('russian')) | set(stopwords.words('english'))
STOP_WORDS = STOP_WORDS | set(['лента', 'новость', 'риа', 'тасс', 'редакция'])

def get_lower(text):
    return str(text).lower().strip()

def remove_punctuation(text):
    return ''.join([c if c.isalpha() or c in ['-',"'"] else ' ' for c in text])

@lru_cache(maxsize=None)
def get_word_normal_form(word):
    return ''.join(mystem.lemmatize(word)).strip().replace('ё', 'е').strip('-')

def lemmatize_words(text):
    res = []
    for word in text.split():
        norm_form = get_word_normal_form(word)
        if len(norm_form) > 2 and norm_form not in STOP_WORDS:
            res.append(norm_form)
    return ' '.join(res)

TEXT_PIPELINE = Pipeline(get_lower, remove_punctuation, lemmatize_words)

In [156]:
%%time
data.text = data.text.progress_apply(TEXT_PIPELINE)
data.title = data.title.apply(lambda x: x.strip())
data['title_norm'] = data.title.progress_apply(TEXT_PIPELINE)

100%|██████████| 219/219 [00:00<00:00, 7541.98it/s]

CPU times: user 556 ms, sys: 76 ms, total: 632 ms
Wall time: 1.06 s


In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 

In [158]:
trainX = data['title_norm'] + ' ' + data.text
trainX = np.swapaxes(np.vstack([trainX.values, data.url.values]), 0, 1)

In [159]:
%%time
tfidf_vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1,2), lowercase=False).fit(trainX[:,0])

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 120 ms


In [160]:
len(tfidf_vectorizer.vocabulary_)

1174

In [161]:
tfidf_matrix = tfidf_vectorizer.transform(trainX[:,0])

In [162]:
cosines = []
for tfidf_news in tfidf_matrix:
    cosine = cosine_similarity(tfidf_news, tfidf_matrix)
    cosines.append(cosine.tolist()[0])

In [163]:
COS_THRESHOLD = 0.5
themes = [ -1 for _ in range(len(cosines)) ]
themes_ids = [ [] for _ in range(len(cosines)) ]
curr_theme = 0
for v, theme in enumerate(themes):
    if theme == -1:
        curr_theme += 1
        Q = []
        Q.append(v)
        themes[v] = curr_theme
        themes_ids[curr_theme].append(v)
        while Q:
            curr_v = Q.pop(0)
            for u, cos in enumerate(cosines[curr_v]):
                if cos >= COS_THRESHOLD and themes[u] == -1:
                    themes[u] = curr_theme
                    themes_ids[curr_theme].append(u)
                    Q.append(u)

In [164]:
themes_ids

[[],
 [0, 21, 38, 167],
 [1, 13, 146, 158, 194, 103, 176],
 [2],
 [3],
 [4, 175],
 [5, 90, 185, 172, 182],
 [6],
 [7],
 [8, 184],
 [9],
 [10],
 [11, 82, 186],
 [12],
 [14, 51, 196, 43],
 [15, 17, 34, 41, 200],
 [16],
 [18],
 [19],
 [20, 28, 208, 74],
 [22],
 [23],
 [24, 39, 64, 142, 156, 66],
 [25],
 [26, 47, 114],
 [27],
 [29, 86, 207],
 [30, 204],
 [31],
 [32],
 [33, 105, 130, 206],
 [35],
 [36],
 [37],
 [40, 177, 202],
 [42],
 [44, 59],
 [45],
 [46],
 [48, 195],
 [49],
 [50],
 [52],
 [53, 201],
 [54],
 [55],
 [56],
 [57],
 [58],
 [60],
 [61],
 [62],
 [63],
 [65],
 [67],
 [68, 81],
 [69],
 [70],
 [71],
 [72],
 [73],
 [75],
 [76, 188],
 [77],
 [78],
 [79],
 [80],
 [83],
 [84],
 [85, 98],
 [87],
 [88, 183, 191],
 [89],
 [91, 154],
 [92],
 [93],
 [94],
 [95],
 [96],
 [97],
 [99],
 [100, 179],
 [101],
 [102],
 [104],
 [106],
 [107],
 [108],
 [109],
 [110],
 [111],
 [112, 133],
 [113],
 [115],
 [116],
 [117],
 [118],
 [119],
 [120],
 [121],
 [122],
 [123],
 [124],
 [125],
 [126],
 [127],


In [165]:
groups = sorted(themes_ids, key=lambda x: -len(x))

In [166]:
for i, group in enumerate(groups):
    if len(group) < 3:
        break
    print('Topic', i)
    for id_ in group:
        print(data.title[id_],data.url[id_])
    print()

Topic 0
В Госдуме отозвали законопроект о праве полиции стрелять в толпе https://www.novayagazeta.ru/news/2017/04/12/130668-v-gosdume-otozvali-zakonoproekt-o-prave-politsii-strelyat-v-tolpe
«Единая Россия» рекомендовала отозвать законопроект о праве полиции стрелять в толпе https://www.novayagazeta.ru/news/2017/04/12/130656-edinaya-rossiya-rekomendovala-otozvat-zakonoproekt-o-prave-politsii-strelyat-v-tolpe
ЕР посоветовала отозвать разрешающий полиции стрелять по толпе законопроект https://lenta.ru/news/2017/04/12/nepodderzhat/
Авторы отозвали законопроект о применении полицией оружия в толпе http://www.vedomosti.ru/politics/news/2017/04/12/685339-politsiei-oruzhiya
«Единая Россия» порекомендовала отозвать законопроект о применении полицией оружия в толпе http://www.vedomosti.ru/politics/news/2017/04/12/685286-otozvat-zakonoproekt
Песков отреагировал на идею разрешить полицейским стрелять по толпе https://lenta.ru/news/2017/04/12/grazhdanskie/
Песков не стал комментировать законопроект